In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-monthly.csv || wget  https://github.com/heiseacademy/ml-python/raw/main/02-ml-intro/heise-monthly.csv")

In [ ]:
import pandas as pd
heise_monthly = pd.read_csv("heise-monthly.csv", parse_dates=["month"], index_col="month")
heise_monthly

In [ ]:
heise_monthly.describe()

In [ ]:
heise_monthly["count"].plot()

In [ ]:
heise_monthly["tokens"].plot()

In [ ]:
!pip install eurostat

In [ ]:
import eurostat
df = eurostat.get_data_df("ei_bsco_m")

In [ ]:
df

In [ ]:
df = df.rename(columns={"geo\\TIME_PERIOD": "country"})

In [ ]:
from datetime import datetime
df.columns = [datetime.strptime(f + "-01", "%Y-%m-%d")
                if f.startswith("20") or f.startswith("19") else f for f in df.columns]

In [ ]:
bs_de = df[(df["indic"] == "BS-CSMCI") & (df["s_adj"] == "SA") & (df["country"] == "DE")]
bs_de = bs_de[[c for c in bs_de.columns 
                     if (isinstance(c, datetime) and c.year>=2000)]]
bs_de

In [ ]:
bs_de.T.plot()

In [ ]:
indic_country = df[df["s_adj"] == "SA"].pivot(index="country", columns="indic", values=datetime(2021, 6, 1))
indic_country

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
sns.heatmap(indic_country)

In [ ]:
# Jahre ab 2000
columns_2000 = [c for c in df.columns if isinstance(c, datetime) and (c.year>=2000)]

de = df[(df["s_adj"] == "SA") & (df["country"] == "DE")].set_index("indic")[columns_2000].transpose()
de

In [ ]:
sns.jointplot(x=de["BS-SFSH"], y=de["BS-CSMCI"], scatter=False, kind="reg")

In [ ]:
sns.pairplot(de)

In [ ]:
from scipy.stats import linregress
from itertools import permutations
res = []
for (i1, i2) in permutations(de.columns, 2):
    res.append({ "Indikator 1": i1, "Indikator 2": i2, "Pearson R": linregress(de[i1], de[i2]).rvalue})

In [ ]:
pd.DataFrame(res).sort_values("Pearson R")